In [0]:
dbutils.library.restartPython()

In [0]:
# Imports
import pandas as pd
import numpy as np
import json
import io
import requests
from collections import Counter
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
print('✅ Imports loaded')

✅ Imports loaded


In [0]:
# Azure settings & Data Loading
import io
import requests

storage_account = 'lab94290'
container = 'airbnb'
sas_token = 'sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D'
base_url = f'https://{storage_account}.blob.core.windows.net/{container}'

parquet_file = 'airbnb_1_12_parquet/part-00000-tid-1637687860512127859-6b9a9b3d-9a1b-4463-b4a9-bd5a0f7f348f-91-1-c000.snappy.parquet'
url = f'{base_url}/{parquet_file}?{sas_token}'

print('📥 Downloading data from Azure...')
response = requests.get(url)
df = pd.read_parquet(io.BytesIO(response.content))
print(f'✅ Data loaded: {len(df):,} rows, {len(df.columns)} columns')

📥 Downloading data from Azure...
✅ Data loaded: 216,065 rows, 50 columns


In [0]:
# Parse functions
def parse_reviews_text(reviews_str):
    if pd.isna(reviews_str): return []
    try:
        reviews = json.loads(reviews_str)
        return [r.replace('<br/>', ' ').strip() for r in reviews if r] if isinstance(reviews, list) else []
    except: return []

def parse_category_ratings(rating_str):
    if pd.isna(rating_str): return {}
    try:
        ratings = json.loads(rating_str) if isinstance(rating_str, str) else rating_str
        return {item['name']: float(item['value']) for item in ratings if 'name' in item} if isinstance(ratings, list) else {}
    except: return {}

def extract_host_id(seller_info):
    try:
        if pd.isna(seller_info): return 'unknown'
        data = json.loads(seller_info) if isinstance(seller_info, str) else seller_info
        return str(data.get('seller_id', 'unknown'))
    except: return 'unknown'

# Process
df_sample = df.sample(n=min(30000, len(df)), random_state=42).copy()
df_sample['host_id'] = df_sample['seller_info'].apply(extract_host_id)
df_sample['reviews_list'] = df_sample['reviews'].apply(parse_reviews_text)
df_sample['ratings_dict'] = df_sample['category_rating'].apply(parse_category_ratings)
df_sample['cleanliness_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Cleanliness'))
df_sample['accuracy_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Accuracy'))
df_sample['checkin_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Check-in'))
df_sample['communication_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Communication'))
df_sample['location_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Location'))
df_sample['value_rating'] = df_sample['ratings_dict'].apply(lambda x: x.get('Value'))
df_sample['review_count'] = df_sample['reviews_list'].apply(len)

# Filter to target locations
target_locations = pd.read_csv('scraped_cities.csv')
target_cities = set(target_locations['city'].str.lower().str.strip().tolist())

def matches_target_location(row):
    loc = str(row.get('location', '')).lower() if pd.notna(row.get('location')) else ""
    for city in target_cities:
        if city in loc:
            return True
    return False

df_filtered = df_sample[df_sample.apply(matches_target_location, axis=1)].copy()

# Explode reviews
reviews_exploded = df_filtered[df_filtered['review_count'] > 0][[
    'property_id', 'listing_name', 'host_id', 'location',
    'reviews_list', 'cleanliness_rating', 'accuracy_rating', 'checkin_rating',
    'communication_rating', 'location_rating', 'value_rating'
]].explode('reviews_list').reset_index(drop=True)
reviews_exploded = reviews_exploded.rename(columns={'reviews_list': 'review_text'})
reviews_exploded = reviews_exploded[reviews_exploded['review_text'].notna() & (reviews_exploded['review_text'].str.len() > 10)].reset_index(drop=True)
reviews_exploded['review_id'] = range(len(reviews_exploded))

print(f'✅ Ready: {len(reviews_exploded):,} reviews from {reviews_exploded["host_id"].nunique():,} hosts')

✅ Ready: 183,425 reviews from 11,312 hosts


In [0]:
# ============================================
# LOAD LIGHTWEIGHT SENTIMENT MODEL
# ============================================
from transformers import pipeline

print("📦 Loading sentiment model...")
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1,  # CPU
    truncation=True,
    max_length=256  # Shorter to save memory
)
print("✅ Model loaded!")

2026-01-29 11:27:35.669937: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 11:27:35.830184: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 11:27:36.457530: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-29 11:27:41.861701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


📦 Loading sentiment model...


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

✅ Model loaded!


In [0]:
# ============================================
# SENTIMENT ANALYSIS - MEMORY SAFE VERSION
# ============================================
from tqdm import tqdm
import gc

def analyze_sentiment_safe(texts, batch_size=32):
    """Analyze sentiment with memory management"""
    results = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Analyzing"):
        batch = [str(t)[:256] for t in texts[i:i+batch_size]]  # Truncate
        
        try:
            batch_results = sentiment_analyzer(batch)
            for r in batch_results:
                score = r['score'] if r['label'] == 'POSITIVE' else -r['score']
                results.append({'label': r['label'], 'score': round(score, 4)})
        except:
            for _ in batch:
                results.append({'label': 'NEUTRAL', 'score': 0.0})
        
        # Clear memory every 1000 batches
        if (i // batch_size) % 1000 == 0:
            gc.collect()
    
    return results

# Run on ALL 106k reviews (smaller batches = less memory)
print(f"🔄 Analyzing {len(reviews_exploded):,} reviews...")


results = analyze_sentiment_safe(reviews_exploded['review_text'].tolist(), batch_size=32)

reviews_exploded['sentiment_label'] = [r['label'] for r in results]
reviews_exploded['sentiment_score'] = [r['score'] for r in results]

print(f"\n✅ Done!")
print(reviews_exploded['sentiment_label'].value_counts())

🔄 Analyzing 183,425 reviews...
   Estimated time: ~15-20 minutes



Analyzing:   7%|▋    

*** WARNING: max output size exceeded, skipping output. ***

Analyzing: 100%|██████████| 5733/5733 [2:44:06<00:00,  1.72s/it]



✅ Done!
POSITIVE    171404
NEGATIVE     12021
Name: sentiment_label, dtype: int64


In [0]:
from collections import Counterimport re# ============================================# MULTILINGUAL ISSUE PATTERNS (EN + DE + FR)# ============================================# Supports: English, German (Deutsch), French (Français)ISSUE_PATTERNS = {    # ============ NEGATIVE ISSUES ============        # WiFi & Tech    "wifi_issues": [        # English        r"\bwifi slow\b", r"\bwifi didn'?t work\b", r"\bno wifi\b", r"\binternet slow\b", r"\bwifi bad\b", r"\bweak wifi\b", r"\bwifi issues\b", r"\bwifi keeps disconnecting\b", r"\bno internet\b", r"\bpoor wifi\b",        # German        r"\bwlan schlecht\b", r"\bkein wlan\b", r"\bkein internet\b", r"\bwlan funktioniert nicht\b", r"\bwlan langsam\b", r"\binternet langsam\b", r"\bwlan probleme\b",        # French        r"\bpas de wifi\b", r"\bwifi lent\b", r"\bpas d'internet\b", r"\binternet lent\b", r"\bwifi ne marche pas\b", r"\bmauvais wifi\b"    ],    "tv_issues": [        r"\btv didn'?t work\b", r"\bno tv\b", r"\btv broken\b", r"\bcouldn'?t use tv\b", r"\bnetflix didn'?t work\b",        # German        r"\bfernseher kaputt\b", r"\bkein fernseher\b", r"\btv funktioniert nicht\b",        # French        r"\btélé cassée\b", r"\bpas de télé\b", r"\btv ne marche pas\b"    ],        # Cleanliness    "dirty_general": [        # English        r"\bdirty\b", r"\bnot clean\b", r"\bunclean\b", r"\bfilthy\b", r"\bdusty\b", r"\bneeds cleaning\b", r"\bwasn'?t clean\b",        # German        r"\bschmutzig\b", r"\bdreckig\b", r"\bnicht sauber\b", r"\bunsauber\b", r"\bverschmutzt\b", r"\bstaubig\b", r"\bmuss gereinigt werden\b", r"\bwar nicht sauber\b", r"\bungepflegt\b",        # French        r"\bsale\b", r"\bpas propre\b", r"\bmalpropre\b", r"\bpoussiéreux\b", r"\bcrasseux\b", r"\bn'était pas propre\b", r"\bbesoin de nettoyage\b"    ],    "dirty_bathroom": [        r"\bdirty bathroom\b", r"\bbathroom not clean\b", r"\bmold\b", r"\bmildew\b", r"\btoilet dirty\b", r"\bshower dirty\b",        # German        r"\bbadezimmer schmutzig\b", r"\bschmutziges bad\b", r"\bschimmel\b", r"\btoilette schmutzig\b", r"\bdusche schmutzig\b", r"\bbad dreckig\b", r"\bbad nicht sauber\b",        # French        r"\bsalle de bain sale\b", r"\btoilettes sales\b", r"\bmoisissure\b", r"\bdouche sale\b", r"\bwc sale\b"    ],    "dirty_kitchen": [        r"\bdirty kitchen\b", r"\bdirty dishes\b", r"\bkitchen not clean\b", r"\bgreasy\b",        # German        r"\bküche schmutzig\b", r"\bschmutziges geschirr\b", r"\bküche nicht sauber\b", r"\bfettig\b", r"\bküche dreckig\b",        # French        r"\bcuisine sale\b", r"\bvaisselle sale\b", r"\bgraisseux\b", r"\bcuisine pas propre\b"    ],    "dirty_bedding": [        r"\bdirty sheets\b", r"\bstained sheets\b", r"\bdirty towels\b", r"\bstained towels\b", r"\bdirty pillows\b",        # German        r"\bschmutzige bettwäsche\b", r"\bfleckige laken\b", r"\bschmutzige handtücher\b", r"\bschmutzige kissen\b", r"\bbettwäsche dreckig\b",        # French        r"\bdraps sales\b", r"\bdraps tachés\b", r"\bserviettes sales\b", r"\boreillers sales\b"    ],    "bugs_pests": [        # English        r"\bbugs\b", r"\binsects\b", r"\bants\b", r"\bcockroach", r"\broaches\b", r"\bspiders\b", r"\bmosquito", r"\bmice\b", r"\brats\b", r"\bbed bugs\b", r"\bflies\b",        # German        r"\bungeziefer\b", r"\binsekten\b", r"\bameisen\b", r"\bkakerlak", r"\bschabe\b", r"\bspinnen\b", r"\bmücken\b", r"\bmäuse\b", r"\bratten\b", r"\bwanzen\b", r"\bbettwanzen\b", r"\bfliegen\b",        # French        r"\binsectes\b", r"\bfourmis\b", r"\bcafard\b", r"\baraignées\b", r"\bmoustiques\b", r"\bsouris\b", r"\brats\b", r"\bpunaises\b", r"\bmouches\b"    ],    "smell_issues": [        # English        r"\bbad smell\b", r"\bsmelled bad\b", r"\bodor\b", r"\bmusty\b", r"\bsmoke smell\b", r"\bsmelly\b", r"\bstinky\b", r"\bcigarette smell\b", r"\bmold smell\b",        # German        r"\bschlechter geruch\b", r"\bgestank\b", r"\briecht schlecht\b", r"\bmuffig\b", r"\brauchgeruch\b", r"\bstinkt\b", r"\bzigarettengeruch\b", r"\bgeruch\b", r"\bübler geruch\b",        # French        r"\bmauvaise odeur\b", r"\bsentait mauvais\b", r"\bodeur\b", r"\bmoisi\b", r"\bodeur de fumée\b", r"\bpuant\b", r"\bodeur de cigarette\b"    ],    "hair_found": [        r"\bhair everywhere\b", r"\bfound hair\b", r"\bhair in bathroom\b", r"\bhair on sheets\b",        # German        r"\bhaare überall\b", r"\bhaare gefunden\b", r"\bhaare im bad\b",        # French        r"\bcheveux partout\b", r"\bcheveux trouvés\b"    ],        # Noise    "noise_street": [        r"\bstreet noise\b", r"\btraffic noise\b", r"\bnoisy street\b", r"\bloud traffic\b", r"\bcars outside\b",        # German        r"\bstraßenlärm\b", r"\bverkehrslärm\b", r"\blaute straße\b", r"\bautos draußen\b",        # French        r"\bbruit de la rue\b", r"\bbruit du trafic\b", r"\brue bruyante\b", r"\bvoitures dehors\b"    ],    "noise_neighbors": [        r"\bloud neighbors\b", r"\bnoisy neighbors\b", r"\bneighbor noise\b", r"\bthin walls\b", r"\bcould hear neighbors\b", r"\bpeople upstairs\b",        # German        r"\blaute nachbarn\b", r"\bnachbarn laut\b", r"\bdünne wände\b", r"\bnachbarn gehört\b", r"\bleute oben\b",        # French        r"\bvoisins bruyants\b", r"\bmurs fins\b", r"\bentendre les voisins\b"    ],    "noise_general": [        # English        r"\bnoisy\b", r"\btoo loud\b", r"\bcouldn'?t sleep.{0,20}noise\b", r"\bnot quiet\b",        # German        r"\blaut\b", r"\bzu laut\b", r"\blärm\b", r"\bnicht ruhig\b", r"\bgeräusche\b", r"\blärmig\b", r"\bhellhörig\b", r"\bkonnte nicht schlafen\b",        # French        r"\bbruyant\b", r"\btrop bruyant\b", r"\bbruit\b", r"\bpas calme\b", r"\bn'ai pas pu dormir\b"    ],    "noise_inside": [        r"\bloud ac\b", r"\bnoisy appliances\b", r"\bloud fridge\b", r"\bnoisy pipes\b",        # German        r"\blaute klimaanlage\b", r"\blauter kühlschrank\b", r"\blaute rohre\b",        # French        r"\bclim bruyante\b", r"\bfrigo bruyant\b"    ],        # Temperature & Comfort    "too_hot": [        # English        r"\btoo hot\b", r"\bno ac\b", r"\bac didn'?t work\b", r"\bac broken\b", r"\bno air conditioning\b", r"\bsweltering\b", r"\bno fan\b",        # German        r"\bzu heiß\b", r"\bzu warm\b", r"\bkeine klimaanlage\b", r"\bklima funktioniert nicht\b", r"\bklima kaputt\b", r"\bkein ventilator\b", r"\bschwül\b",        # French        r"\btrop chaud\b", r"\bpas de clim\b", r"\bclim ne marche pas\b", r"\bclim cassée\b", r"\bétouffant\b"    ],    "too_cold": [        # English        r"\btoo cold\b", r"\bno heating\b", r"\bfreezing\b", r"\bheater broken\b", r"\bno heat\b", r"\bcold at night\b",        # German        r"\bzu kalt\b", r"\bkeine heizung\b", r"\beiskalt\b", r"\bheizung kaputt\b", r"\bfrieren\b", r"\bnachts kalt\b",        # French        r"\btrop froid\b", r"\bpas de chauffage\b", r"\bgelé\b", r"\bchauffage cassé\b", r"\bfroid la nuit\b"    ],    "uncomfortable_bed": [        r"\buncomfortable bed\b", r"\bbad mattress\b", r"\bhard mattress\b", r"\bsoft mattress\b", r"\blumpy mattress\b", r"\bbed too small\b", r"\bback hurt\b",        # German        r"\bunbequemes bett\b", r"\bschlechte matratze\b", r"\bharte matratze\b", r"\bweiche matratze\b", r"\bbett zu klein\b", r"\brückenschmerzen\b",        # French        r"\blit inconfortable\b", r"\bmauvais matelas\b", r"\bmatelas dur\b", r"\bmatelas mou\b", r"\blit trop petit\b", r"\bmal au dos\b"    ],    "uncomfortable_pillows": [        r"\bbad pillows\b", r"\bflat pillows\b", r"\buncomfortable pillows\b", r"\bno pillows\b",        # German        r"\bschlechte kissen\b", r"\bflache kissen\b", r"\bunbequeme kissen\b", r"\bkeine kissen\b",        # French        r"\bmauvais oreillers\b", r"\boreillers plats\b", r"\bpas d'oreillers\b"    ],        # Water & Plumbing    "hot_water_issues": [        r"\bno hot water\b", r"\bcold water only\b", r"\bwater pressure low\b", r"\bhot water ran out\b", r"\bwater not hot\b",        # German        r"\bkein heißes wasser\b", r"\bkein warmwasser\b", r"\bwasserdruck niedrig\b", r"\bwasser nicht heiß\b",        # French        r"\bpas d'eau chaude\b", r"\beau froide seulement\b", r"\bpression d'eau faible\b"    ],    "plumbing_issues": [        r"\btoilet broken\b", r"\btoilet wouldn'?t flush\b", r"\bclogged drain\b", r"\bsink clogged\b", r"\bleaking\b", r"\bleak\b", r"\bdripping\b",        # German        r"\btoilette kaputt\b", r"\btoilette spült nicht\b", r"\babfluss verstopft\b", r"\bundicht\b", r"\btropft\b", r"\bleck\b",        # French        r"\btoilettes cassées\b", r"\bchasse d'eau ne marche pas\b", r"\bbouché\b", r"\bfuite\b", r"\bgoutte\b"    ],    "shower_issues": [        r"\bshower didn'?t work\b", r"\bweak shower\b", r"\bshower broken\b", r"\bno water pressure\b",        # German        r"\bdusche funktioniert nicht\b", r"\bdusche kaputt\b", r"\bschwacher wasserdruck\b",        # French        r"\bdouche ne marche pas\b", r"\bdouche cassée\b", r"\bpas de pression\b"    ],        # Check-in & Access    "checkin_problems": [        r"\bcheck-?in problem\b", r"\bcheck-?in difficult\b", r"\bcouldn'?t check in\b", r"\blate check-?in\b", r"\bearly check-?in denied\b",        # German        r"\bcheck-?in problem\b", r"\bcheck-?in schwierig\b", r"\bspäter check-?in\b",        # French        r"\bproblème.{0,5}arrivée\b", r"\barrivée difficile\b"    ],    "hard_to_find": [        r"\bhard to find\b", r"\bcouldn'?t find\b", r"\bwrong address\b", r"\bconfusing directions\b", r"\bgot lost\b", r"\bno signs\b",        # German        r"\bschwer zu finden\b", r"\bnicht gefunden\b", r"\bfalsche adresse\b", r"\bverirrt\b", r"\bkeine schilder\b",        # French        r"\bdifficile à trouver\b", r"\bpas trouvé\b", r"\bmauvaise adresse\b", r"\bperdu\b"    ],    "key_issues": [        r"\bkey didn'?t work\b", r"\blockbox problem\b", r"\bcouldn'?t get in\b", r"\blocked out\b", r"\bwrong code\b", r"\bkey broken\b",        # German        r"\bschlüssel funktioniert nicht\b", r"\bschlüsselbox problem\b", r"\bausgesperrt\b", r"\bfalscher code\b",        # French        r"\bclé ne marche pas\b", r"\benfermé dehors\b", r"\bmauvais code\b"    ],    "parking_issues": [        r"\bno parking\b", r"\bparking difficult\b", r"\bparking far\b", r"\bcouldn'?t park\b", r"\bparking expensive\b",        # German        r"\bkein parkplatz\b", r"\bparken schwierig\b", r"\bparkplatz weit\b", r"\bparken teuer\b",        # French        r"\bpas de parking\b", r"\bstationnement difficile\b", r"\bparking loin\b", r"\bparking cher\b"    ],        # Host & Communication    "host_unresponsive": [        r"\bhost didn'?t respond\b", r"\bno response\b", r"\bslow response\b", r"\bcouldn'?t reach host\b", r"\bhost unavailable\b", r"\bignored messages\b",        # German        r"\bgastgeber antwortet nicht\b", r"\bkeine antwort\b", r"\blangsame antwort\b", r"\bgastgeber nicht erreichbar\b",        # French        r"\bhôte ne répond pas\b", r"\bpas de réponse\b", r"\bréponse lente\b", r"\bhôte injoignable\b"    ],    "host_rude": [        r"\bhost rude\b", r"\bhost unfriendly\b", r"\bhost unhelpful\b", r"\bbad host\b", r"\bterrible host\b",        # German        r"\bgastgeber unhöflich\b", r"\bgastgeber unfreundlich\b", r"\bschlechter gastgeber\b",        # French        r"\bhôte impoli\b", r"\bhôte désagréable\b", r"\bmauvais hôte\b"    ],    "wrong_expectations": [        r"\bnot as described\b", r"\bphotos misleading\b", r"\bdifferent from photos\b", r"\bfalse advertising\b", r"\bnot what expected\b", r"\bsmaller than photos\b",        # German        r"\bnicht wie beschrieben\b", r"\bfotos irreführend\b", r"\banders als fotos\b", r"\bkleiner als fotos\b", r"\bnicht wie erwartet\b",        # French        r"\bpas comme décrit\b", r"\bphotos trompeuses\b", r"\bdifférent des photos\b", r"\bplus petit que les photos\b"    ],        # Value & Price    "overpriced": [        # English        r"\boverpriced\b", r"\btoo expensive\b", r"\bnot worth the price\b", r"\bexpensive for what\b", r"\brip off\b", r"\bwaste of money\b",        # German        r"\bzu teuer\b", r"\büberteuert\b", r"\bpreis nicht wert\b", r"\bgeldverschwendung\b", r"\babzocke\b",        # French        r"\btrop cher\b", r"\bsurprix\b", r"\bne vaut pas le prix\b", r"\barnaque\b", r"\bgaspillage d'argent\b"    ],    "hidden_fees": [        r"\bhidden fees\b", r"\bextra charges\b", r"\bsurprise fees\b", r"\bcleaning fee too high\b",        # German        r"\bversteckte kosten\b", r"\bzusätzliche kosten\b", r"\breinigungsgebühr zu hoch\b",        # French        r"\bfrais cachés\b", r"\bfrais supplémentaires\b", r"\bfrais de ménage trop élevés\b"    ],        # Safety & Security    "safety_concerns": [        r"\bdidn'?t feel safe\b", r"\bunsafe\b", r"\bsketchy area\b", r"\bdangerous neighborhood\b", r"\bsecurity concerns\b",        # German        r"\bnicht sicher gefühlt\b", r"\bunsicher\b", r"\bgefährliche gegend\b", r"\bsicherheitsbedenken\b",        # French        r"\bpas en sécurité\b", r"\bdangereux\b", r"\bquartier dangereux\b", r"\bproblèmes de sécurité\b"    ],    "broken_lock": [        r"\block broken\b", r"\bdoor didn'?t lock\b", r"\bno lock\b", r"\bsecurity issue\b",        # German        r"\bschloss kaputt\b", r"\btür schließt nicht\b", r"\bkein schloss\b",        # French        r"\bserrure cassée\b", r"\bporte ne ferme pas\b", r"\bpas de verrou\b"    ],        # Amenities Missing    "missing_amenities": [        r"\bno towels\b", r"\bno toilet paper\b", r"\bno soap\b", r"\bno shampoo\b", r"\bnothing provided\b",        # German        r"\bkeine handtücher\b", r"\bkein toilettenpapier\b", r"\bkeine seife\b", r"\bkein shampoo\b", r"\bnichts vorhanden\b",        # French        r"\bpas de serviettes\b", r"\bpas de papier toilette\b", r"\bpas de savon\b", r"\bpas de shampoing\b", r"\brien fourni\b"    ],    "kitchen_issues": [        r"\bno cooking supplies\b", r"\bno pots\b", r"\bno pans\b", r"\bkitchen not equipped\b", r"\bno utensils\b", r"\bno dishes\b",        # German        r"\bkeine kochtöpfe\b", r"\bkeine pfannen\b", r"\bküche nicht ausgestattet\b", r"\bkein besteck\b", r"\bkein geschirr\b",        # French        r"\bpas de casseroles\b", r"\bpas de poêles\b", r"\bcuisine pas équipée\b", r"\bpas de couverts\b", r"\bpas de vaisselle\b"    ],    "laundry_issues": [        r"\bwasher broken\b", r"\bdryer broken\b", r"\bno laundry\b", r"\bwashing machine didn'?t work\b",        # German        r"\bwaschmaschine kaputt\b", r"\btrockner kaputt\b", r"\bkeine waschmaschine\b",        # French        r"\bmachine à laver cassée\b", r"\bsèche-linge cassé\b", r"\bpas de machine à laver\b"    ],        # Space Issues    "too_small": [        # English        r"\btoo small\b", r"\btiny\b", r"\bcramped\b", r"\bsmaller than expected\b", r"\bno space\b",        # German        r"\bzu klein\b", r"\bwinzig\b", r"\beng\b", r"\bkleiner als erwartet\b", r"\bkein platz\b", r"\bbeengt\b",        # French        r"\btrop petit\b", r"\bminuscule\b", r"\bexigu\b", r"\bplus petit que prévu\b", r"\bpas d'espace\b"    ],    "stairs_access": [        r"\btoo many stairs\b", r"\bno elevator\b", r"\bhard to access\b", r"\bnot accessible\b",        # German        r"\bzu viele treppen\b", r"\bkein aufzug\b", r"\bkein fahrstuhl\b", r"\bschwer zugänglich\b", r"\bnicht barrierefrei\b",        # French        r"\btrop d'escaliers\b", r"\bpas d'ascenseur\b", r"\bdifficile d'accès\b", r"\bpas accessible\b"    ],        # ============ POSITIVE THEMES ============        # Location    "great_location": [        r"\bgreat location\b", r"\bperfect location\b", r"\bamazing location\b", r"\bbest location\b", r"\bideal location\b", r"\bexcellent location\b", r"\bfantastic location\b", r"\bprime location\b",        # German        r"\btolle lage\b", r"\bperfekte lage\b", r"\bsuper lage\b", r"\bausgezeichnete lage\b", r"\bideale lage\b", r"\bhervorragende lage\b",        # French        r"\bsuper emplacement\b", r"\bemplacement parfait\b", r"\bexcellent emplacement\b", r"\btrès bien situé\b"    ],    "walkable": [        r"\bwalking distance\b", r"\bwalkable\b", r"\bclose to everything\b", r"\bsteps away\b", r"\beasy walk\b", r"\bwalk to\b",        # German        r"\bfußläufig\b", r"\bzu fuß erreichbar\b", r"\bin der nähe\b",        # French        r"\bà pied\b", r"\bà distance de marche\b", r"\bproche de tout\b"    ],    "near_transport": [        r"\bnear metro\b", r"\bnear subway\b", r"\bnear bus\b", r"\bclose to train\b", r"\beasy transport\b", r"\bpublic transport\b",        # German        r"\bnähe metro\b", r"\bnähe u-bahn\b", r"\bgute anbindung\b", r"\böffentliche verkehrsmittel\b",        # French        r"\bprès du métro\b", r"\bprès du bus\b", r"\btransports en commun\b"    ],    "quiet_area": [        r"\bquiet area\b", r"\bpeaceful\b", r"\bquiet neighborhood\b", r"\bquiet street\b", r"\bvery quiet\b", r"\btranquil\b",        # German        r"\bruhige gegend\b", r"\bruhig\b", r"\bfriedlich\b", r"\bstille straße\b", r"\bsehr ruhig\b",        # French        r"\bquartier calme\b", r"\bcalme\b", r"\bpaisible\b", r"\brue calme\b", r"\btrès calme\b"    ],    "central": [        r"\bcentral location\b", r"\bin the center\b", r"\bheart of the city\b", r"\bdowntown\b", r"\bcity center\b",        # German        r"\bzentrale lage\b", r"\bim zentrum\b", r"\bstadtmitte\b", r"\binnenstadt\b",        # French        r"\bcentre-ville\b", r"\bau centre\b", r"\bcoeur de la ville\b"    ],    "good_neighborhood": [        r"\bnice neighborhood\b", r"\bgreat neighborhood\b", r"\bsafe neighborhood\b", r"\blovely area\b", r"\bnice area\b", r"\bbeautiful neighborhood\b",        # German        r"\bschöne gegend\b", r"\bgute nachbarschaft\b", r"\bsichere gegend\b",        # French        r"\bbeau quartier\b", r"\bquartier sûr\b", r"\bjoli quartier\b"    ],        # Cleanliness    "very_clean": [        r"\bvery clean\b", r"\bspotless\b", r"\bimmaculate\b", r"\bsuper clean\b", r"\bsparkling clean\b", r"\bextremely clean\b", r"\bperfectly clean\b", r"\bpristine\b",        # German        r"\bsehr sauber\b", r"\bblitzsauber\b", r"\bmakellos\b", r"\bsuper sauber\b", r"\btadelos sauber\b",        # French        r"\btrès propre\b", r"\bimpeccable\b", r"\bnickel\b", r"\bsuper propre\b"    ],        # Host    "helpful_host": [        r"\bhelpful host\b", r"\bhost was helpful\b", r"\bhost helped\b", r"\baccommodating host\b", r"\bhost went above\b",        # German        r"\bhilfreicher gastgeber\b", r"\bgastgeber war hilfreich\b", r"\bsehr hilfsbereit\b",        # French        r"\bhôte serviable\b", r"\bhôte très aidant\b"    ],    "great_host": [        r"\bgreat host\b", r"\bamazing host\b", r"\bwonderful host\b", r"\bbest host\b", r"\bfantastic host\b", r"\blovely host\b", r"\bexcellent host\b",        # German        r"\btoller gastgeber\b", r"\bsuper gastgeber\b", r"\bwunderbarer gastgeber\b", r"\bbester gastgeber\b",        # French        r"\bsuper hôte\b", r"\bhôte formidable\b", r"\bexcellent hôte\b"    ],    "responsive_host": [        r"\bresponsive host\b", r"\bquick response\b", r"\bfast response\b", r"\bresponded quickly\b", r"\bquick to respond\b", r"\bimmediate response\b", r"\balways available\b",        # German        r"\bschnelle antwort\b", r"\bsofort geantwortet\b", r"\bimmer erreichbar\b",        # French        r"\bréponse rapide\b", r"\bréactif\b", r"\btoujours disponible\b"    ],    "friendly_host": [        r"\bfriendly host\b", r"\bwelcoming host\b", r"\bwarm welcome\b", r"\bhost was friendly\b", r"\bhost was kind\b", r"\bgracious host\b",        # German        r"\bfreundlicher gastgeber\b", r"\bherzlicher empfang\b", r"\bnetter gastgeber\b",        # French        r"\bhôte sympathique\b", r"\baccueil chaleureux\b", r"\bhôte gentil\b"    ],    "great_communication": [        r"\bgreat communication\b", r"\beasy communication\b", r"\bclear communication\b", r"\bgood communication\b", r"\bexcellent communication\b",        # German        r"\bgute kommunikation\b", r"\btolle kommunikation\b",        # French        r"\bbonne communication\b", r"\bcommunication facile\b"    ],        # Views & Aesthetics    "great_view": [        r"\bgreat view\b", r"\bamazing view\b", r"\bbeautiful view\b", r"\bstunning view\b", r"\bincredible view\b", r"\bgorgeous view\b", r"\bwonderful view\b", r"\bocean view\b", r"\bcity view\b", r"\bmountain view\b",        # German        r"\btolle aussicht\b", r"\bschöne aussicht\b", r"\bwunderbare aussicht\b", r"\bblick auf\b",        # French        r"\bbelle vue\b", r"\bvue magnifique\b", r"\bvue superbe\b"    ],    "beautiful_place": [        r"\bbeautiful place\b", r"\bgorgeous\b", r"\bstunning\b", r"\blovely decor\b", r"\bbeautifully decorated\b", r"\bstylish\b", r"\bcharming\b",        # German        r"\bwunderschön\b", r"\btoll eingerichtet\b", r"\bstilvoll\b", r"\bcharmant\b",        # French        r"\bmagnifique\b", r"\bjoliment décoré\b", r"\bélégant\b", r"\bcharmant\b"    ],        # Value    "good_value": [        r"\bgreat value\b", r"\bgood value\b", r"\bworth the price\b", r"\bgood price\b", r"\bexcellent value\b", r"\bfair price\b", r"\breasonable price\b", r"\baffordable\b", r"\bworth every penny\b", r"\bgreat deal\b",        # German        r"\bgutes preis-leistungs-verhältnis\b", r"\bpreiswert\b", r"\bgünstiger preis\b", r"\bfairer preis\b",        # French        r"\bbon rapport qualité-prix\b", r"\bprix raisonnable\b", r"\bbonne affaire\b"    ],        # Amenities    "well_equipped": [        r"\bwell equipped\b", r"\bhad everything\b", r"\bfully equipped\b", r"\ball amenities\b", r"\beverything we needed\b", r"\bfully stocked\b", r"\bwell stocked\b",        # German        r"\bgut ausgestattet\b", r"\balles vorhanden\b", r"\bvoll ausgestattet\b",        # French        r"\bbien équipé\b", r"\btout le nécessaire\b", r"\bentièrement équipé\b"    ],    "great_kitchen": [        r"\bgreat kitchen\b", r"\bfull kitchen\b", r"\bwell equipped kitchen\b", r"\bloved the kitchen\b", r"\bkitchen had everything\b",        # German        r"\btolle küche\b", r"\bgut ausgestattete küche\b",        # French        r"\bsuper cuisine\b", r"\bcuisine bien équipée\b"    ],    "nice_bathroom": [        r"\bnice bathroom\b", r"\bgreat bathroom\b", r"\bbeautiful bathroom\b", r"\bclean bathroom\b", r"\bmodern bathroom\b", r"\bspacious bathroom\b",        # German        r"\bschönes badezimmer\b", r"\bsauberes bad\b", r"\bmodernes bad\b",        # French        r"\bbelle salle de bain\b", r"\bsalle de bain propre\b"    ],    "good_wifi": [        r"\bfast wifi\b", r"\bgreat wifi\b", r"\bwifi worked\b", r"\bstrong wifi\b", r"\bgood internet\b", r"\bexcellent wifi\b",        # German        r"\bschnelles wlan\b", r"\bgutes internet\b", r"\bwlan funktioniert\b",        # French        r"\bwifi rapide\b", r"\bbon wifi\b", r"\binternet rapide\b"    ],    "good_bed": [        r"\bcomfortable bed\b", r"\bgreat bed\b", r"\bslept well\b", r"\bcomfy bed\b", r"\bgood mattress\b", r"\bbest sleep\b",        # German        r"\bbequemes bett\b", r"\bgut geschlafen\b", r"\bgute matratze\b",        # French        r"\blit confortable\b", r"\bbien dormi\b", r"\bbon matelas\b"    ],    "good_shower": [        r"\bgreat shower\b", r"\bamazing shower\b", r"\bgood water pressure\b", r"\bhot shower\b", r"\bnice shower\b",        # German        r"\btolle dusche\b", r"\bguter wasserdruck\b", r"\bheiße dusche\b",        # French        r"\bsuper douche\b", r"\bbonne pression d'eau\b"    ],    "pool_great": [        r"\bgreat pool\b", r"\bloved the pool\b", r"\bnice pool\b", r"\bbeautiful pool\b", r"\bpool was perfect\b",        # German        r"\btoller pool\b", r"\bschöner pool\b",        # French        r"\bbelle piscine\b", r"\bpiscine géniale\b"    ],    "good_ac": [        r"\bac worked great\b", r"\bgood ac\b", r"\bnice and cool\b", r"\bac was perfect\b", r"\bgood air conditioning\b",        # German        r"\bklima funktioniert gut\b", r"\bgute klimaanlage\b",        # French        r"\bclim parfaite\b", r"\bbonne climatisation\b"    ],        # Comfort    "very_comfortable": [        r"\bvery comfortable\b", r"\bsuper comfortable\b", r"\bextremely comfortable\b", r"\bso comfortable\b", r"\bcozy\b", r"\bhomey\b", r"\bfelt like home\b",        # German        r"\bsehr bequem\b", r"\bsehr gemütlich\b", r"\bgemütlich\b", r"\bwie zuhause\b",        # French        r"\btrès confortable\b", r"\bcosy\b", r"\bcomme chez soi\b"    ],    "spacious": [        r"\bspacious\b", r"\blots of space\b", r"\bplenty of room\b", r"\broomy\b", r"\bbig apartment\b", r"\blarge space\b",        # German        r"\bgeräumig\b", r"\bviel platz\b", r"\bgroße wohnung\b",        # French        r"\bspacieux\b", r"\bbeaucoup d'espace\b", r"\bgrand appartement\b"    ],        # Experience    "will_return": [        r"\bwill be back\b", r"\bcome back\b", r"\breturn\b", r"\bstay again\b", r"\bbook again\b", r"\bvisit again\b", r"\bdefinitely return\b",        # German        r"\bkommen wieder\b", r"\bwieder buchen\b", r"\bgerne wieder\b", r"\bwürde wiederkommen\b",        # French        r"\breviendrai\b", r"\bre-réserver\b", r"\brevenir\b"    ],    "highly_recommend": [        r"\bhighly recommend\b", r"\bdefinitely recommend\b", r"\bwould recommend\b", r"\bstrongly recommend\b", r"\bmust stay\b", r"\bdon'?t hesitate\b",        # German        r"\bsehr empfehlenswert\b", r"\bsehr zu empfehlen\b", r"\bkann ich empfehlen\b", r"\bunbedingt buchen\b",        # French        r"\bfortement recommandé\b", r"\bje recommande\b", r"\bn'hésitez pas\b"    ],    "exceeded_expectations": [        r"\bexceeded expectations\b", r"\bbetter than expected\b", r"\babove and beyond\b", r"\bblown away\b", r"\bpleasantly surprised\b",        # German        r"\bübertroffen\b", r"\bbesser als erwartet\b", r"\bpositiv überrascht\b",        # French        r"\bdépassé les attentes\b", r"\bmieux que prévu\b", r"\bagréablement surpris\b"    ],    "perfect_stay": [        r"\bperfect stay\b", r"\bamazing stay\b", r"\bwonderful stay\b", r"\bfantastic stay\b", r"\bincredible stay\b", r"\bbest stay\b", r"\bloved our stay\b",        # German        r"\bperfekter aufenthalt\b", r"\btoller aufenthalt\b", r"\bwunderbarer aufenthalt\b",        # French        r"\bséjour parfait\b", r"\bséjour incroyable\b", r"\bsuper séjour\b"    ],        # Check-in    "easy_checkin": [        r"\beasy check-?in\b", r"\bsmooth check-?in\b", r"\bsimple check-?in\b", r"\bself check-?in\b", r"\bseamless check-?in\b",        # German        r"\beinfacher check-?in\b", r"\breibungsloser check-?in\b",        # French        r"\barrivée facile\b", r"\bcheck-?in simple\b"    ],    "good_instructions": [        r"\bclear instructions\b", r"\bgood directions\b", r"\beasy to find\b", r"\bwell explained\b", r"\bdetailed instructions\b",        # German        r"\bklare anweisungen\b", r"\bgute beschreibung\b", r"\bleicht zu finden\b",        # French        r"\binstructions claires\b", r"\bfacile à trouver\b", r"\bbien expliqué\b"    ],        # Family/Groups    "family_friendly": [        r"\bgreat for families\b", r"\bkids loved\b", r"\bfamily friendly\b", r"\bperfect for kids\b", r"\bgreat for children\b",        # German        r"\bfamilienfreundlich\b", r"\bkinder waren begeistert\b", r"\btoll für familien\b",        # French        r"\bparfait pour les familles\b", r"\benfants adoré\b", r"\bfamilial\b"    ],    "good_for_groups": [        r"\bgreat for groups\b", r"\bperfect for group\b", r"\bfit everyone\b", r"\benough space for everyone\b",        # German        r"\btoll für gruppen\b", r"\bplatz für alle\b",        # French        r"\bparfait pour les groupes\b", r"\bplace pour tout le monde\b"    ]}

🔍 Extracting themes with fixed patterns...

✅ Theme extraction complete!

📊 TOP POSITIVE THEMES:
   ✅ will_return: 22,060
   ✅ highly_recommend: 20,601
   ✅ very_comfortable: 20,531
   ✅ very_clean: 18,692
   ✅ great_location: 17,551
   ✅ walkable: 15,527
   ✅ well_equipped: 14,460
   ✅ spacious: 13,128
   ✅ quiet_area: 12,955
   ✅ great_host: 9,639
   ✅ beautiful_place: 8,388
   ✅ perfect_stay: 7,152
   ✅ central: 6,985
   ✅ responsive_host: 5,889
   ✅ great_view: 3,659
   ✅ good_value: 3,433
   ✅ good_instructions: 2,599
   ✅ great_communication: 2,465
   ✅ friendly_host: 2,120
   ✅ good_neighborhood: 1,669
   ✅ exceeded_expectations: 1,603
   ✅ good_bed: 1,513
   ✅ great_kitchen: 1,271
   ✅ helpful_host: 1,147
   ✅ family_friendly: 980
   ✅ pool_great: 923
   ✅ near_transport: 849
   ✅ easy_checkin: 572
   ✅ good_shower: 315
   ✅ good_wifi: 304
   ✅ nice_bathroom: 295
   ✅ good_ac: 111

📊 TOP NEGATIVE THEMES (Issues):
   ❌ dirty_general: 1,725
   ❌ noise_general: 1,424
   ❌ bugs_pes

In [0]:
# ============================================
# STEP 7: AGGREGATE BY HOST
# ============================================
from collections import Counter

print("📊 Aggregating data by host...")

# Group by host
host_data = []

for host_id, group in reviews_exploded.groupby('host_id'):
    # Basic info
    listings = group['listing_name'].unique().tolist()
    locations = group['location'].unique().tolist()
    total_reviews = len(group)
    
    # Sentiment stats
    avg_sentiment = group['sentiment_score'].mean()
    positive_count = (group['sentiment_label'] == 'POSITIVE').sum()
    negative_count = (group['sentiment_label'] == 'NEGATIVE').sum()
    
    # Ratings (from Airbnb's own ratings)
    avg_cleanliness = group['cleanliness_rating'].mean()
    avg_communication = group['communication_rating'].mean()
    avg_checkin = group['checkin_rating'].mean()
    avg_location = group['location_rating'].mean()
    avg_value = group['value_rating'].mean()
    
    # Collect all themes
    all_positive = [t for themes in group['positive_themes'] for t in themes]
    all_negative = [t for themes in group['negative_themes'] for t in themes]
    
    # Count themes
    positive_counts = Counter(all_positive)
    negative_counts = Counter(all_negative)
    
    # Format advantages (top 5 with counts)
    advantages = []
    for theme, count in positive_counts.most_common(5):
        if count >= 2:  # Only if mentioned 2+ times
            advantages.append(f"{theme.replace('_', ' ').title()} ({count}x)")
    
    # Format disadvantages (top 5 with counts)
    disadvantages = []
    for theme, count in negative_counts.most_common(5):
        if count >= 1:  # Even 1 complaint is worth noting
            disadvantages.append(f"{theme.replace('_', ' ').title()} ({count}x)")
    
    host_data.append({
        'host_id': host_id,
        'listings': "; ".join(listings[:3]) + ("..." if len(listings) > 3 else ""),
        'locations': "; ".join(list(set(locations))[:2]),
        'total_reviews': total_reviews,
        'positive_reviews': positive_count,
        'negative_reviews': negative_count,
        'avg_sentiment_score': round(avg_sentiment, 3),
        'avg_cleanliness': round(avg_cleanliness, 2) if pd.notna(avg_cleanliness) else None,
        'avg_communication': round(avg_communication, 2) if pd.notna(avg_communication) else None,
        'avg_checkin': round(avg_checkin, 2) if pd.notna(avg_checkin) else None,
        'avg_location': round(avg_location, 2) if pd.notna(avg_location) else None,
        'avg_value': round(avg_value, 2) if pd.notna(avg_value) else None,
        'advantages': "; ".join(advantages) if advantages else "No specific themes identified",
        'disadvantages': "; ".join(disadvantages) if disadvantages else "No issues identified",
        'positive_theme_counts': dict(positive_counts),
        'negative_theme_counts': dict(negative_counts)
    })

# Create DataFrame
host_summary = pd.DataFrame(host_data)

# Sort by total reviews
host_summary = host_summary.sort_values('total_reviews', ascending=False).reset_index(drop=True)

print(f"\n✅ Aggregated data for {len(host_summary):,} hosts")
print(f"\n📋 Sample (Top 5 hosts by review count):")
print(host_summary[['host_id', 'total_reviews', 'avg_sentiment_score', 'advantages', 'disadvantages']].head())

📊 Aggregating data by host...

✅ Aggregated data for 11,312 hosts

📋 Sample (Top 5 hosts by review count):
     host_id  ...                                      disadvantages
0   61137746  ...  Dirty General (5x); Bugs Pests (5x); Noise Gen...
1   93929916  ...  Plumbing Issues (3x); Dirty General (3x); Too ...
2  151397468  ...  Dirty General (6x); Dirty Kitchen (1x); Dirty ...
3  122380652  ...  Bugs Pests (2x); Too Cold (1x); Too Small (1x)...
4  121681645  ...  Dirty General (4x); Bugs Pests (2x); Hard To F...

[5 rows x 5 columns]


In [0]:
# ============================================
# EXTRACT SUGGESTIONS FROM ACTUAL ARTICLES
# ============================================
import json

# Load knowledge base
print("📚 Loading knowledge base from articles...")
with open('scraping_results_full_english.json', 'r') as f:
    knowledge_base_raw = json.load(f)

# Extract ALL tips from articles
all_tips = []
for article in knowledge_base_raw:
    if article.get("status") != "success":
        continue
    
    content = article.get("content", {})
    title = content.get("page_title", "")
    url = article.get("url", "")
    
    # Get tips from list_items (most actionable content)
    list_items = content.get("list_items", [])
    for item in list_items:
        # Filter out navigation/footer items, keep actionable tips
        if len(item) > 30 and len(item) < 500:
            skip_words = ['privacy', 'footer', 'help center', 'airbnb your', 
                         'site footer', 'support', 'hosting', 'careers', 
                         'newsroom', 'investors', 'gift cards', '©']
            if not any(skip in item.lower() for skip in skip_words):
                all_tips.append({
                    'tip': item,
                    'source_title': title,
                    'source_url': url
                })
    
    # Also get tips from paragraphs
    paragraphs = content.get("paragraphs", [])
    for para in paragraphs:
        if len(para) > 50 and len(para) < 400:
            skip_words = ['privacy', 'footer', 'help center', 'may have changed']
            if not any(skip in para.lower() for skip in skip_words):
                all_tips.append({
                    'tip': para,
                    'source_title': title,
                    'source_url': url
                })

print(f"✅ Extracted {len(all_tips)} tips from articles")

# Preview some tips
print("\n📋 Sample tips from knowledge base:")
for tip in all_tips[:5]:
    print(f"   • {tip['tip'][:100]}...")

📚 Loading knowledge base from articles...
✅ Extracted 1191 tips from articles

📋 Sample tips from knowledge base:
   • How to take great photos for your listing...
   • Clean and declutter. Show guests exactly what they can expect. “It’s surprising how many people forg...
   • Choose features to highlight. What will guests love about your place? What do they need to know to a...
   • Make a list. Plan to capture each room and area guests can use from various perspectives. For exampl...
   • Find the best light. Soft, natural light gives photos a warm and welcoming tone. Plan to take indoor...


In [0]:
# ============================================
# MAP ISSUES TO RELEVANT ARTICLE TIPS
# ============================================

# Define keywords to match issues to tips
ISSUE_KEYWORDS = {
    "dirty_general": ["clean", "declutter", "tidy", "spotless", "cleaning"],
    "dirty_bathroom": ["bathroom", "toilet", "shower", "towel", "mirror"],
    "dirty_kitchen": ["kitchen", "countertop", "dishes", "cooking", "utensils"],
    "dirty_bedding": ["sheet", "bed", "pillow", "linen", "blanket", "comforter"],
    "bugs_pests": ["pest", "bug", "insect", "clean", "hygiene"],
    "smell_issues": ["smell", "odor", "fresh", "ventilation", "air"],
    "hair_found": ["clean", "hair", "bathroom", "sheets"],
    
    "noise_general": ["noise", "quiet", "sound", "peaceful"],
    "noise_street": ["noise", "street", "location", "neighborhood"],
    "noise_neighbors": ["noise", "wall", "neighbor", "sound"],
    "noise_inside": ["appliance", "ac", "air conditioning"],
    
    "too_hot": ["ac", "air conditioning", "cool", "temperature", "fan"],
    "too_cold": ["heat", "heating", "warm", "temperature", "blanket"],
    "uncomfortable_bed": ["bed", "mattress", "pillow", "sleep", "comfortable"],
    "uncomfortable_pillows": ["pillow", "sleep", "comfortable"],
    
    "hot_water_issues": ["water", "hot water", "shower", "plumbing"],
    "plumbing_issues": ["plumbing", "water", "toilet", "drain", "leak"],
    "shower_issues": ["shower", "water", "bathroom", "pressure"],
    
    "checkin_problems": ["check-in", "checkin", "arrival", "access", "instructions"],
    "hard_to_find": ["find", "direction", "address", "location", "instructions", "map"],
    "key_issues": ["key", "lock", "lockbox", "access", "entry", "door"],
    "parking_issues": ["parking", "car", "garage", "driveway"],
    
    "host_unresponsive": ["response", "respond", "communication", "message", "contact", "available"],
    "host_rude": ["communication", "guest", "hospitality", "friendly", "welcome"],
    
    "wrong_expectations": ["photo", "picture", "description", "accurate", "expect", "listing"],
    
    "overpriced": ["price", "pricing", "value", "discount", "smart pricing", "competitive"],
    "hidden_fees": ["fee", "price", "cleaning fee", "cost", "transparent"],
    
    "safety_concerns": ["safe", "safety", "security", "lock", "secure"],
    "broken_lock": ["lock", "door", "security", "key", "secure"],
    
    "missing_amenities": ["amenity", "amenities", "provide", "essential", "stock", "supply"],
    "kitchen_issues": ["kitchen", "cooking", "utensil", "pan", "pot", "appliance"],
    "laundry_issues": ["laundry", "washer", "dryer", "washing"],
    
    "too_small": ["space", "size", "photo", "accurate", "description", "expect"],
    "stairs_access": ["stair", "elevator", "access", "floor", "accessibility"],
    
    "wifi_issues": ["wifi", "internet", "connection", "network"],
    "tv_issues": ["tv", "television", "streaming", "entertainment"]
}

def find_relevant_tips(issue, all_tips, max_tips=3):
    """Find tips from knowledge base that match an issue"""
    if issue not in ISSUE_KEYWORDS:
        return []
    
    keywords = ISSUE_KEYWORDS[issue]
    relevant_tips = []
    
    for tip_data in all_tips:
        tip_lower = tip_data['tip'].lower()
        # Count how many keywords match
        match_count = sum(1 for kw in keywords if kw in tip_lower)
        if match_count >= 1:
            relevant_tips.append({
                'tip': tip_data['tip'],
                'source': tip_data['source_title'],
                'match_score': match_count
            })
    
    # Sort by match score and return top tips
    relevant_tips.sort(key=lambda x: x['match_score'], reverse=True)
    return relevant_tips[:max_tips]

# Build complete suggestion mapping from articles
print("🔗 Mapping issues to article tips...")
issue_to_tips = {}

for issue in ISSUE_KEYWORDS.keys():
    tips = find_relevant_tips(issue, all_tips, max_tips=4)
    issue_to_tips[issue] = tips
    
# Show what we mapped
print(f"\n✅ Mapped {len(issue_to_tips)} issues to article tips")
print("\n📋 Sample mappings:")
for issue in ["dirty_general", "wifi_issues", "overpriced"]:
    print(f"\n   ❌ {issue.upper()}:")
    for tip in issue_to_tips.get(issue, [])[:2]:
        print(f"      → {tip['tip'][:80]}...")
        print(f"        (Source: {tip['source'][:50]})")

🔗 Mapping issues to article tips...

✅ Mapped 36 issues to article tips

📋 Sample mappings:

   ❌ DIRTY_GENERAL:
      → Clean and declutter. Show guests exactly what they can expect. “It’s surprising ...
        (Source: How to take great photos for your listing - Resour)
      → The total price guests pay. Understanding what guests pay can help you set a com...
        (Source: Review your price - Resource Center - Airbnb)

   ❌ WIFI_ISSUES:
      → Wifi: Provide the wifi network and password at your home or location....
        (Source: Using quick replies to save time - Resource Center)
      → Sharing your space can bring you lasting memories, worldwide connections, and ca...
        (Source: Set and customize nightly pricing - Airbnb Help Ce)

   ❌ OVERPRICED:
      → You can develop and evolve your Smart Pricing strategy over time. This includes ...
        (Source: What is Smart Pricing? – Resource Centre – Airbnb)
      → Check price tips and similar listings: These tools help

In [0]:
# ============================================
# CHECK KNOWLEDGE BASE STATISTICS
# ============================================

print("📊 KNOWLEDGE BASE ANALYSIS")
print("=" * 60)

# Count articles
total_articles = len(knowledge_base_raw)
successful_articles = sum(1 for a in knowledge_base_raw if a.get("status") == "success")

print(f"\n📚 Articles: {successful_articles} successful / {total_articles} total")

# Count tips per article
article_stats = []
for article in knowledge_base_raw:
    if article.get("status") != "success":
        continue
    
    content = article.get("content", {})
    title = content.get("page_title", "Unknown")[:50]
    list_items = len(content.get("list_items", []))
    paragraphs = len(content.get("paragraphs", []))
    
    article_stats.append({
        'title': title,
        'list_items': list_items,
        'paragraphs': paragraphs,
        'total': list_items + paragraphs
    })

# Summary
total_list_items = sum(a['list_items'] for a in article_stats)
total_paragraphs = sum(a['paragraphs'] for a in article_stats)

print(f"\n📝 Raw content extracted:")
print(f"   - List items: {total_list_items}")
print(f"   - Paragraphs: {total_paragraphs}")
print(f"   - Total raw: {total_list_items + total_paragraphs}")

print(f"\n✅ After filtering (removing navigation/footer): {len(all_tips)} useful tips")
print(f"   Average per article: {len(all_tips) / successful_articles:.1f} tips")

# Show articles with most tips
print(f"\n📋 Top 10 articles by content:")
article_stats_sorted = sorted(article_stats, key=lambda x: x['total'], reverse=True)
for a in article_stats_sorted[:10]:
    print(f"   • {a['title']}... ({a['total']} items)")

# Show articles with fewest tips
print(f"\n📋 Articles with least content:")
for a in article_stats_sorted[-5:]:
    print(f"   • {a['title']}... ({a['total']} items)")

📊 KNOWLEDGE BASE ANALYSIS

📚 Articles: 73 successful / 74 total

📝 Raw content extracted:
   - List items: 2075
   - Paragraphs: 586
   - Total raw: 2661

✅ After filtering (removing navigation/footer): 1191 useful tips
   Average per article: 16.3 tips

📋 Top 10 articles by content:
   • How to become a Superhost - Resource Center - Airb... (63 items)
   • Add house rules to a listing - Airbnb Help Center... (63 items)
   • Ground rules for home guests - Airbnb Help Center... (58 items)
   • Getting ready for your photoshoot - Airbnb... (57 items)
   • Add check-in and checkout instructions to listings... (57 items)
   • Set and customize nightly pricing - Airbnb Help Ce... (56 items)
   • Airbnb’s 5-step enhanced cleaning process - Airbnb... (56 items)
   • Reviews for homes - Airbnb Help Center... (55 items)
   • Ratings for homes - Airbnb Help Center... (52 items)
   • AirCover for guests - Airbnb Help Center... (52 items)

📋 Articles with least content:
   • How to provide self ch

In [0]:
# ============================================
# GENERATE SUGGESTIONS FOR EACH HOST
# ============================================

def generate_host_suggestions(negative_theme_counts, issue_to_tips):
    """Generate suggestions based on host's specific issues using article tips"""
    if not negative_theme_counts:
        return "No specific issues identified. Keep up the great work!"
    
    suggestions = []
    
    # Sort issues by frequency (most common first)
    sorted_issues = sorted(negative_theme_counts.items(), key=lambda x: x[1], reverse=True)
    
    for issue, count in sorted_issues[:3]:  # Focus on top 3 issues
        if issue in issue_to_tips and issue_to_tips[issue]:
            # Get the best tip for this issue
            best_tip = issue_to_tips[issue][0]
            tip_text = best_tip['tip']
            
            # Truncate if too long
            if len(tip_text) > 200:
                tip_text = tip_text[:197] + "..."
            
            issue_name = issue.replace('_', ' ').title()
            suggestions.append(f"[{issue_name}]: {tip_text}")
    
    if suggestions:
        return " || ".join(suggestions)
    else:
        return "Review guest feedback and focus on cleanliness, communication, and accurate listing details."

def get_suggestion_sources(negative_theme_counts, issue_to_tips):
    """Get article sources for the suggestions"""
    if not negative_theme_counts:
        return ""
    
    sources = set()
    sorted_issues = sorted(negative_theme_counts.items(), key=lambda x: x[1], reverse=True)
    
    for issue, count in sorted_issues[:3]:
        if issue in issue_to_tips and issue_to_tips[issue]:
            sources.add(issue_to_tips[issue][0]['source'][:70])
    
    return " | ".join(sources)

# Apply to all hosts
print("💡 Generating suggestions for each host...")

host_summary['suggestions'] = host_summary['negative_theme_counts'].apply(
    lambda x: generate_host_suggestions(x, issue_to_tips)
)

host_summary['suggestion_sources'] = host_summary['negative_theme_counts'].apply(
    lambda x: get_suggestion_sources(x, issue_to_tips)
)

print(f"✅ Suggestions generated for {len(host_summary):,} hosts")

# Preview results
print(f"\n{'='*70}")
print("📋 SAMPLE HOST REPORTS")
print('='*70)

for i, row in host_summary.head(3).iterrows():
    print(f"\n🏠 HOST: {row['host_id']}")
    print(f"📍 Location: {row['locations']}")
    print(f"📊 Reviews: {row['total_reviews']} total ({row['positive_reviews']} pos / {row['negative_reviews']} neg)")
    print(f"⭐ Avg Sentiment: {row['avg_sentiment_score']}")
    print(f"\n✅ ADVANTAGES:\n   {row['advantages']}")
    print(f"\n❌ DISADVANTAGES:\n   {row['disadvantages']}")
    print(f"\n💡 SUGGESTIONS:\n   {row['suggestions'][:400]}")
    print(f"\n📚 Sources:\n   {row['suggestion_sources']}")
    print("-" * 70)

💡 Generating suggestions for each host...
✅ Suggestions generated for 11,312 hosts

📋 SAMPLE HOST REPORTS

🏠 HOST: 61137746
📍 Location: Davenport, Florida, United States; Kissimmee, Florida, United States
📊 Reviews: 456 total (422 pos / 34 neg)
⭐ Avg Sentiment: 0.852

✅ ADVANTAGES:
   Will Return (74x); Very Clean (60x); Spacious (50x); Highly Recommend (41x); Great Location (33x)

❌ DISADVANTAGES:
   Dirty General (5x); Bugs Pests (5x); Noise General (3x); Hard To Find (1x); Too Hot (1x)

💡 SUGGESTIONS:
   [Dirty General]: Clean and declutter. Show guests exactly what they can expect. “It’s surprising how many people forget to clear their spaces of random objects before taking photos,” says Jeff, a photographer and S... || [Bugs Pests]: Cleanliness: Did the home meet our standards? (It should be free of health hazards like mold and pests. It should also be free of extensive dust, pet dander, dirty d

📚 Sources:
   How to become a Superhost - Resource Center - Airbnb | How to take grea

In [0]:
# ============================================
# EXPORT TO CSV
# ============================================

# Select and rename columns for final CSV
final_csv = host_summary[[
    'host_id',
    'listings',
    'locations',
    'total_reviews',
    'positive_reviews',
    'negative_reviews',
    'avg_sentiment_score',
    'avg_cleanliness',
    'avg_communication',
    'avg_checkin',
    'avg_location',
    'avg_value',
    'advantages',
    'disadvantages',
    'suggestions',
    'suggestion_sources'
]].copy()

# Rename columns for clarity
final_csv.columns = [
    'Host_ID',
    'Listings',
    'Locations',
    'Total_Reviews',
    'Positive_Reviews',
    'Negative_Reviews',
    'Avg_Sentiment_Score',
    'Avg_Cleanliness_Rating',
    'Avg_Communication_Rating',
    'Avg_Checkin_Rating',
    'Avg_Location_Rating',
    'Avg_Value_Rating',
    'Advantages',
    'Disadvantages',
    'Suggestions',
    'Suggestion_Sources'
]

# Export to CSV
output_filename = 'host_review_intelligence_report.csv'
final_csv.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"✅ CSV exported: {output_filename}")
print(f"   Total hosts: {len(final_csv):,}")
print(f"   Columns: {list(final_csv.columns)}")

# Preview
print(f"\n📋 Preview (first 3 rows):")
print(final_csv.head(3).to_string())

✅ CSV exported: host_review_intelligence_report.csv
   Total hosts: 11,312
   Columns: ['Host_ID', 'Listings', 'Locations', 'Total_Reviews', 'Positive_Reviews', 'Negative_Reviews', 'Avg_Sentiment_Score', 'Avg_Cleanliness_Rating', 'Avg_Communication_Rating', 'Avg_Checkin_Rating', 'Avg_Location_Rating', 'Avg_Value_Rating', 'Advantages', 'Disadvantages', 'Suggestions', 'Suggestion_Sources']

📋 Preview (first 3 rows):
     Host_ID                                                                                                                                                                                 Listings                                                                                 Locations  Total_Reviews  Positive_Reviews  Negative_Reviews  Avg_Sentiment_Score  Avg_Cleanliness_Rating  Avg_Communication_Rating  Avg_Checkin_Rating  Avg_Location_Rating  Avg_Value_Rating                                                                                              Advantages          